In [64]:
%matplotlib inline

import glob
import matplotlib.pyplot as plt
import nilearn.image, nilearn.masking, nilearn.plotting
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import cross_val_predict

In [65]:
behav_dir = '/Volumes/clmnlab/TM/behav_data/'
roi_dir = '/Volumes/clmnlab/TM/fMRI_data/masks/Fan/Fan280/'
stats_dir = '/Volumes/clmnlab/TM/fMRI_data/stats/Reg6_MVPA2_IM_VWM/'
preproc_dir = '/Volumes/clmnlab/TM/fMRI_data/preproc_data/'
subj_list = ["TML04_PILOT", "TML05_PILOT", "TML06_PILOT", "TML07_PILOT", 
             "TML08_PILOT", "TML09_PILOT", "TML10_PILOT", "TML11_PILOT"]

#subj_list = ["TML04_PILOT"]

### Neural data of touching the comparement group

- Freq.index: 비교군이 먼저 오는지, 나중에 오는지  
- LSSout: 한 trial 안에서 beta를 5개씩 구했음  
- 비교군을 만질 때의 neural data를 사용하므로, 비교군이 먼저 온 경우 첫번째 beta, 비교군이 나중에 온 경우 세번째 beta를 선택해야 함(get_label_index_others)  

In [66]:
def get_label_index_others(subj, run):
## labeling betas of others indices ##
## others : frequencies except center frequencies ##
    df = pd.read_csv(behav_dir + '%s/%s.r0%d.freq_class.dat' % (subj, subj, run), sep='\t', index_col=0)
    others = [i * 5 if x == 1 else i * 5 + 2 for i, x in enumerate(df['Freq.index'])]
    return others

In [67]:
def get_label_index_centers(subj, run):
## labeling betas of centers indices ##
## centers : center frequencies ##
    df = pd.read_csv(behav_dir + '%s/%s.r0%d.freq_class.dat' % (subj, subj, run), sep='\t', index_col=0)
    centers = [i * 5 + 2 if x == 1 else i * 5 for i, x in enumerate(df['Freq.index'])]
    return centers

In [68]:
def load_beta_image_others(subj, run):
## load nilearn image ##
    img = nilearn.image.load_img(stats_dir + '%s/r0%d.LSSout.nii.gz' % (subj, run))
    other_indices = get_label_index_others(subj, run)    
    other_img = nilearn.image.index_img(img, other_indices)
    return other_img

In [69]:
def load_beta_image_centers(subj, run):
## load nilearn image ##
    img = nilearn.image.load_img(stats_dir + '%s/r0%d.LSSout.nii.gz' % (subj, run))
    center_indices = get_label_index_centers(subj, run)    
    center_img = nilearn.image.index_img(img, center_indices)
    return center_img

In [70]:
def load_target(subj, run):
# load behavior data and make up them
    df = pd.read_csv(behav_dir + '%s/%s.r0%d.freq_class.dat' % (subj, subj, run), sep='\t', index_col=0)
    result = list(df['Freq.class'])
    return result

### X & y & grouping

In [71]:
### X: neural data / y: answers / group: run number

def get_X_y_group(subj, runs):      
    Xs = [
        load_beta_image_others(subj, run)
        for run in runs
    ]
    
    cXs = [
        load_beta_image_centers(subj, run)
        for run in runs
    ]
    
    ys = [
        load_target(subj, run)
        for run in runs
    ]
    group = [
        i for i, y in enumerate(ys) for j in range(len(y))
    ]
    Xs = nilearn.image.concat_imgs(Xs)
    cXs = nilearn.image.concat_imgs(cXs)
    ys = np.concatenate(ys)
    assert Xs.shape[-1] == ys.shape[0]
    assert ys.shape[0] == len(group)
    return Xs, cXs, ys, group

### ROI Masking
- Fan ROI 사용

In [72]:
def fast_masking(img, roi):
    img_data = img.get_data()
    roi_mask = roi.get_data().astype(bool)
    
    if img_data.shape[:3] != roi_mask.shape:
        raise ValueError('different shape while masking! img=%s and roi=%s' % (img_data.shape, roi_mask.shape))
        
    return img_data[roi_mask, :].T

In [73]:
roi_path = sorted(glob.glob(roi_dir + 'fan.roi.resam.*.nii.gz'))
roi_imgs = []

for path in roi_path:
    roi_imgs.append(nilearn.image.load_img(path))

### Model fitting

In [74]:
svc_clf = LinearSVC()
results = {}
cv = GroupKFold(3)

for subj in subj_list:
    X, cX, y, group = get_X_y_group(subj, [1, 2, 3])
    decisions_o = {}
    decisions_c = {}
    for m, roi in enumerate(roi_imgs):
        roi_index = m+1
        roi_X = fast_masking(roi=roi, img=X)
        roi_cX = fast_masking(roi=roi, img=cX)
        svc_score = cross_validate(svc_clf, roi_X, y, group, cv=cv,
                                   return_train_score=True, return_estimator=True)
        results[subj, roi_index] = np.mean(svc_score['test_score'])
        #decisions[subj, roi_index] = cross_val_predict(svc_clf, roi_X, y, group, cv=GroupKFold(3),
        #                                                         method="decision_function")
        #print(subj, roi_index, end='\r')
        
        decision_boundary_dist_other = []
        decision_boundary_dist_center = []

        for i, (train_idx, test_idx) in enumerate(cv.split(X=roi_X, groups=group, y=y)):
            decision_boundary_dist_other.append(
                list(svc_score['estimator'][i].decision_function(roi_X[test_idx])))
            
            decision_boundary_dist_center.append(
                list(svc_score['estimator'][i].decision_function(roi_cX[test_idx])))
            
        decision_boundary_dist_other = decision_boundary_dist_other[0] + decision_boundary_dist_other[2] + decision_boundary_dist_other[1]
        decision_boundary_dist_center = decision_boundary_dist_center[0] + decision_boundary_dist_center[2] + decision_boundary_dist_center[1]
        
        decisions_o[subj, roi_index] = decision_boundary_dist_other
        decisions_c[subj, roi_index] = decision_boundary_dist_center
    
    df_dec = np.vstack([decisions_o[subj, i+1] for i in range(roi_index)]).T
    df_dec = pd.DataFrame(df_dec)
    df_dec.to_csv("/Volumes/clmnlab/TM/fMRI_data/MVPA/2020319_TM_SVC_Fan_allROI_decision_fn_other_%s.csv" % (subj))
    
    df_dec = np.vstack([decisions_c[subj, i+1] for i in range(roi_index)]).T
    df_dec = pd.DataFrame(df_dec)
    df_dec.to_csv("/Volumes/clmnlab/TM/fMRI_data/MVPA/2020319_TM_SVC_Fan_allROI_decision_fn_center_%s.csv" % (subj))
    
print(subj, end='\r')

/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iteratio

/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iteratio

/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iteratio

/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/jisu/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iteratio

In [75]:
df = pd.DataFrame(columns=['subject', 'roi', 'accu'])
for i,j in zip(results.keys(), results.values()):
    df = df.append({'subject':i[0], 'roi':i[1], 'accu':j}, ignore_index=True)
    
df.head()
df.to_csv("/Volumes/clmnlab/TM/fMRI_data/MVPA/2020319_TM_SVC_Fan_allROI_train_score.csv")